In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

df = pd.read_csv('crime-housing-austin-2015.csv')

# Change percentages to floats so that they can be graphed
df['Ownerunitsaffordabletoaverageteacher'] = df['Ownerunitsaffordabletoaverageteacher'].str.replace('%', '').astype('float')
df['Ownerunitsaffordabletoaveragetechworker'] = df['Ownerunitsaffordabletoaveragetechworker'].str.replace('%', '').astype('float')

# Drop N/A values in dataset for the demographics we're interested in
df.dropna(subset=['Ownerunitsaffordabletoaveragetechworker', 'Ownerunitsaffordabletoaverageteacher'])

# Look at the distribution for housing affordability
sns.distplot(df['Ownerunitsaffordabletoaveragetechworker'], label='tech worker', hist=False)
sns.distplot(df['Ownerunitsaffordabletoaverageteacher'], label='teacher', hist=False)
plt.xlabel('Percent of Affordable Owner Units in the Area of the Crime')
plt.ylabel('Density')
plt.title('Housing Affordability')
plt.legend()
plt.show()

### Interesting Observations:

Most crimes are reported in places where the average tech worker can afford to own a house/unit. That's not terribly surprising, because the average tech worker can probably afford to own a house/unit in most places.

It's strange that there is a spike in places where less than 40% of average tech workers can afford to buy a house.

There are a fair amount of crimes reported in places where less than 40% of teachers can afford to buy a house. 

There are relatively few crimes recorded for places where only about half of tech workers and only about half of teachers can afford to buy a house.

There is a spike in crimes recorded for places where about 70% of teachers can afford to buy a house.

In general, it looks like there are more crimes in more affordable (cheaper) housing areas.

In [ ]:
# Create a column for how much time passes between a crime being reported and a crime being cleared
df['Clearance_Time'] = pd.to_datetime(df['Clearance_Date'], format='%d-%b-%y') - pd.to_datetime(df['Report_Date'], format='%d-%b-%y')
df['Clearance_Time'] = df['Clearance_Time'].dt.days
df = df.dropna(subset=['Clearance_Time', 'Clearance_Status'])

# Make a table for each demographic
# Areas where less than half of tech workers can afford to rent a place
lowAffordability = df[df['Ownerunitsaffordabletoaveragetechworker'] < 50]
# Areas where more than half of teachers can afford to rent a place
highAffordability = df[df['Ownerunitsaffordabletoaverageteacher'] > 50]

totalCrimes = len(lowAffordability) + len(highAffordability)
lowAffordabilityCrimesPercent = len(lowAffordability) / totalCrimes
highAffordabilityCrimesPercent = len(highAffordability) / totalCrimes
display(f"Percent of low affordability crimes: {lowAffordabilityCrimesPercent}")
display(f"Percent of high affordability crimes: {highAffordabilityCrimesPercent}")
display(f"There are {highAffordabilityCrimesPercent - lowAffordabilityCrimesPercent} % more crimes recorded in our high-affordability housing areas than in our low-affordability housing areas")

It looks like there isn't a significant difference in the quantity of crimes recorded for very expensive areas vs very cheap areas. That's somewhat surprising.

## Is there a difference between low-affordability housing areas and high-affordability housing areas in the time it takes for a crime in those areas to be cleared?

In [ ]:
# Make a table for each clearance status in low affordability areas
lowAffordabilityArrest = lowAffordability[lowAffordability['Clearance_Status'] == 'C']
lowAffordabilityException = lowAffordability[lowAffordability['Clearance_Status'] == 'O']
lowAffordabilityNot = lowAffordability[lowAffordability['Clearance_Status'] == 'N']

# Make a table for each clearance status in high affordability areas
highAffordabilityArrest = highAffordability[highAffordability['Clearance_Status'] == 'C']
highAffordabilityException = highAffordability[highAffordability['Clearance_Status'] == 'O']
highAffordabilityNot = highAffordability[highAffordability['Clearance_Status'] == 'N']

# Take the mean of each clearance time by type, and housing affordability
display("Low Affordability areas:")
display(f"mean clearance time when cleared by arrest: {lowAffordabilityArrest['Clearance_Time'].mean()}")
display(f"percent of crimes cleared by arrest: {len(lowAffordabilityArrest) / len(lowAffordability)}")
display(f"mean clearance time when cleared by exception: {lowAffordabilityException['Clearance_Time'].mean()}")
display(f"percent of crimes cleared by exception: {len(lowAffordabilityException) / len(lowAffordability)}")
display(f"mean clearance time when not cleared: {lowAffordabilityNot['Clearance_Time'].mean()}")
display(f"percent of crimes not cleared: {len(lowAffordabilityNot) / len(lowAffordability)}")

display("High Affordability areas:")
display(f"mean clearance time when cleared by arrest: {highAffordabilityArrest['Clearance_Time'].mean()}")
display(f"percent of crimes cleared by arrest: {len(highAffordabilityArrest) / len(highAffordability)}")
display(f"mean clearance time when cleared by exception: {highAffordabilityException['Clearance_Time'].mean()}")
display(f"percent of crimes cleared by exception: {len(highAffordabilityException) / len(highAffordability)}")
display(f"mean clearance time when not cleared: {highAffordabilityNot['Clearance_Time'].mean()}")
display(f"percent of crimes not cleared: {len(highAffordabilityNot) / len(highAffordability)}")

# Compare the clearance time of each kind of status between low and high affordability housing areas
sns.distplot(x=lowAffordabilityArrest['Clearance_Time'], label='low', hist=False)
sns.distplot(x=highAffordabilityArrest['Clearance_Time'], label='high', hist=False)
plt.xlabel('Clearance Time (Days)')
plt.ylabel('Density')
plt.title('Distribution of Clearance Time for Crimes Cleared by Arrest')
plt.legend()
plt.show()

sns.distplot(x=lowAffordabilityException['Clearance_Time'], label='low', hist=False)
sns.distplot(x=highAffordabilityException['Clearance_Time'], label='high', hist=False)
plt.xlabel('Clearance Time (Days)')
plt.ylabel('Density')
plt.title('Distribution of Clearance Time for Crimes Cleared by Exception')
plt.legend()
plt.show()

sns.distplot(x=lowAffordabilityNot['Clearance_Time'], label='low', hist=False)
sns.distplot(x=highAffordabilityNot['Clearance_Time'], label='high', hist=False)
plt.xlabel('Clearance Time (Days)')
plt.ylabel('Density')
plt.title('Distribution of Clearance Time for Crimes Not Cleared')
plt.legend()
plt.show()

It seems that low-affordability housing areas (expensive housing) has lower clearance time than high-affordability housing areas (cheap housing). This could be because crimes being prosecuted in more expensive areas are given a higher priority, or that police departments in those areas have better funding. Let's run a t-test to see if this is statistically significant or not.

In [ ]:
display(stats.ttest_ind(highAffordabilityArrest['Clearance_Time'], lowAffordabilityArrest['Clearance_Time']))
display(stats.ttest_ind(highAffordabilityException['Clearance_Time'], lowAffordabilityException['Clearance_Time']))
display(stats.ttest_ind(highAffordabilityNot['Clearance_Time'], lowAffordabilityNot['Clearance_Time']))

Now compare clearance time overall for each housing group:

In [ ]:
sns.distplot(x=lowAffordability['Clearance_Time'], label='low', hist=False)
sns.distplot(x=highAffordability['Clearance_Time'], label='high', hist=False)
plt.xlabel('Clearance Time (Days)')
plt.ylabel('Density')
plt.title('Distribution of Clearance Time Overall')
plt.legend()
plt.show()

display(stats.ttest_ind(highAffordability['Clearance_Time'], lowAffordability['Clearance_Time']))


## Conclusions:

Vast majority of crimes are not cleared.
Minority of crimes are cleared by exception.

It takes less time for crimes to be cleared in areas with more expensive (low-affordability) housing.